In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
import tqdm

Using TensorFlow backend.


In [2]:
train = pd.read_csv("./train.csv")

In [3]:
train.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [4]:
img = plt.imread("./train/"+train['id'][0])

In [5]:
img.shape

(32, 32, 3)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(train['id'], train['has_cactus'], test_size = 0.1, random_state = 32)

In [7]:
x_train.head()

11878    abd9a552e5fd12f11abc08d70817dccb.jpg
17221    fbd51e7dbb31cee4b2e11ee20cd133ef.jpg
14558    d3de0d511a8b13e260e6d47b1fa242f7.jpg
2785     27147093c1c112bccb7a8c6ab346d512.jpg
8147     74f611a1f3d41313fe35ec75c50e3c15.jpg
Name: id, dtype: object

In [8]:
x_test.head()

11059    9fdc9fc1e21de90bb6fa60c352730874.jpg
7766     6f91f3f61c05efd2d19c8fb8daeb01e3.jpg
4723     436cc6cee6013fd9d92129e8731c9fe9.jpg
4219     3b7099dc3b98cb00fa31be60f8b57f58.jpg
4263     3c229646fbb827820b634191465a8902.jpg
Name: id, dtype: object

In [9]:
y_train.head()

11878    1
17221    0
14558    1
2785     1
8147     1
Name: has_cactus, dtype: int64

In [18]:
xtrain_arr = []
for image in tqdm.tqdm(x_train):
    img = plt.imread("./train/"+image)
    xtrain_arr.append(img)

100%|██████████████████████████████████████████████████████████████████████████| 15750/15750 [00:03<00:00, 4222.69it/s]


In [19]:
xtrain_arr = np.array(xtrain_arr)
print(xtrain_arr.shape)

(15750, 32, 32, 3)


In [17]:
xtest_arr = []

In [20]:
for image in tqdm.tqdm(x_test):
    img = plt.imread("./train/"+image)
    xtest_arr.append(img)

100%|██████████████████████████████████████████████████████████████████████████████| 1750/1750 [00:17<00:00, 98.15it/s]


In [21]:
xtest_arr = np.array(xtest_arr)
print(xtest_arr.shape)

(1750, 32, 32, 3)


In [22]:
xtrain_arr = xtrain_arr.astype('float32')
xtrain_arr = xtrain_arr/255
xtest_arr = xtest_arr.astype('float32')
xtest_arr = xtest_arr/255

In [32]:
from keras.models import Model,Sequential,load_model
from keras.layers import *
from keras import applications
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
from keras.losses import binary_crossentropy, sparse_categorical_crossentropy
from keras import backend as K

In [35]:
from keras.applications.xception import Xception
from keras.layers import *

inputs = Input((32, 32, 3))
base_model = Xception(include_top=False, input_shape=(71, 71, 3))#, weights=None
x = base_model(inputs)
out1 = GlobalMaxPooling2D()(x)
out2 = GlobalAveragePooling2D()(x)
out3 = Flatten()(x)
out = Concatenate(axis=-1)([out1, out2, out3])
out = Dropout(0.5)(out)
out = Dense(256, name="3_")(out)
out = BatchNormalization()(out)
out = Activation("relu")(out)
out = Dense(1, activation="sigmoid", name="3_2")(out)
model = Model(inputs, out)
model.summary()

83689472/83683744 [==============================] - 235s 3us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
xception (Model)                multiple             20861480    input_3[0][0]                    
__________________________________________________________________________________________________
global_max_pooling2d_1 (GlobalM (None, 2048)         0           xception[1][0]                   
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 2048)         0           xception[1][0]                   
__________________________________________

In [36]:
base_model.Trainable=True

set_trainable=False
for layer in base_model.layers:
    layer.trainable = True

In [37]:
model.compile('rmsprop', loss = "binary_crossentropy", metrics=["accuracy"])

In [39]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

batch_size = 128
epochs = 36

filepath="weights_resnet.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
learning_rate_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=4, verbose=1, mode='max', min_delta=0.0, cooldown=0, min_lr=0)
early_stop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=25, verbose=1, mode='max', baseline=None, restore_best_weights=True)

train_datagen = ImageDataGenerator(
    rotation_range=40,
    zoom_range=0.1,
    vertical_flip=True,
    horizontal_flip=True)

train_datagen.fit(xtrain_arr)
history = model.fit_generator(
    train_datagen.flow(xtrain_arr, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=(xtest_arr, y_test),
    callbacks=[learning_rate_reduce, checkpoint] 
)

Epoch 1/36
123/123 [==============================] - 34s 278ms/step - loss: 0.1843 - acc: 0.9419 - val_loss: 0.4412 - val_acc: 0.9480

Epoch 00001: val_acc improved from -inf to 0.94800, saving model to weights_resnet.hdf5
Epoch 2/36
123/123 [==============================] - 12s 100ms/step - loss: 0.0571 - acc: 0.9818 - val_loss: 0.4165 - val_acc: 0.9297

Epoch 00002: val_acc did not improve from 0.94800
Epoch 3/36
123/123 [==============================] - 12s 100ms/step - loss: 0.0714 - acc: 0.9822 - val_loss: 0.1225 - val_acc: 0.9651

Epoch 00003: val_acc improved from 0.94800 to 0.96514, saving model to weights_resnet.hdf5
Epoch 4/36
123/123 [==============================] - 12s 99ms/step - loss: 0.0481 - acc: 0.9889 - val_loss: 0.0114 - val_acc: 0.9960

Epoch 00004: val_acc improved from 0.96514 to 0.99600, saving model to weights_resnet.hdf5
Epoch 5/36
123/123 [==============================] - 12s 98ms/step - loss: 0.0258 - acc: 0.9927 - val_loss: 0.0293 - val_acc: 0.9949

Ep

In [41]:
from sklearn.metrics import confusion_matrix, roc_auc_score
train_pred = model.predict(xtrain_arr, verbose= 1)
valid_pred = model.predict(xtest_arr, verbose= 1)

train_acc = roc_auc_score(np.round(train_pred), y_train)
valid_acc = roc_auc_score(np.round(valid_pred), y_test)

1750/1750 [==============================] - 1s 526us/step


In [42]:
confusion_matrix(np.round(valid_pred), y_test)

array([[ 446,    0],
       [   1, 1303]], dtype=int64)

In [43]:
sample = pd.read_csv('./sample_submission.csv')

In [44]:
test = []
for images in tqdm.tqdm(sample['id']):
    img = plt.imread('./test/' + images)
    test.append(img)

test = np.array(test)
test = test/255

100%|█████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:20<00:00, 194.10it/s]


In [45]:
test_pred = model.predict(test, verbose= 1)

4000/4000 [==============================] - 2s 535us/step


In [46]:
sample['has_cactus'] = test_pred
sample.head()

,id,has_cactus
0,000940378805c44108d287872b2f04ce.jpg,9.999691e-01
1,0017242f54ececa4512b4d7937d1e21e.jpg,9.999769e-01
2,001ee6d8564003107853118ab87df407.jpg,6.044542e-06
3,002e175c3c1e060769475f52182583d0.jpg,2.758825e-07
4,0036e44a7e8f7218e9bc7bf8137e4943.jpg,9.999669e-01


In [47]:
sample.to_csv('sub.csv', index= False)